## A gentle 10-minute primer to Ray AI Runitime (Ray AIR)

© 2019-2022, Anyscale. All Rights Reserved

📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

### Overview

As part of Ray 2.0, Ray AI Runtime (AIR) is an open-source and unified toolkit for building end-to-end simple and scalable ML applications. 

<img src = "https://docs.ray.io/en/latest/_images/ray-air.svg" width="60%" height="30%">

Ray AI Runtime focuses on two functional aspects:
 * It provides scalability by leveraging Ray’s distributed compute layer for ML workloads.
 * It is designed to interoperate with other systems for storage and metadata needs.

Ray AIR consists of five key components:

<img src = "../images/air_ml_workflow.png" width="60%" height="30%">

 * Data processing ([Ray Data](https://docs.ray.io/en/latest/data/dataset.html))
 * Model Training ([Ray Train](https://docs.ray.io/en/latest/train/train.html))
 * Hyperparameter Tuning ([Ray Tune](https://docs.ray.io/en/latest/tune/index.html))
 * Model Serving ([Ray Serve](https://docs.ray.io/en/latest/serve/index.html)).
 * Reinforcement Learning ([Ray RLlib](https://docs.ray.io/en/latest/rllib/index.html))
 
 
📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>
 
### Learning objectives:
  * How to use Ray AIR as a unified toolkit to write an end-to-end ML application in a single Python script
  * Use out-of-box Preprocessors
  * Load model from the best model checkpoint and use for batch inference
  * Deploy best checkpoint model and use for online inference

In [2]:
import logging, os, random, warnings
import ray
import pandas as pd

In [3]:
warnings.filterwarnings("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"

In [4]:
if ray.is_initialized:
    ray.shutdown()
ray.init(logging_level=logging.ERROR)

Python version:,3.8.13
Ray version:,3.0.0.dev0
Dashboard:,http://127.0.0.1:8266


### End-to-end ML stages for a Ray AIR ML application

<img src="../images/ray_air_pipeline.png" width="50%" height="25%">

### 1. Create Ray data from an S3 CSV datasource

In [5]:
dataset = ray.data.read_csv("s3://anonymous@air-example-data/breast_cancer.csv")

# Split data into train and validation.
train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)
test_dataset = valid_dataset.drop_columns(["target"])

Map_Batches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 49.15it/s]


### 2. Use out-of-the-box Preprocessors
This preprocessor is automatically used in the training function to `fit` and `tranform` your datasets for training and validation. You don't have to explicitly call the preprocess before training or inference. Ray AIR toolkit automatically does that for you. 

<img src="../images/data_prep.png" width="50%" height="25%">

We are going to scaler a few features like `mean radius` and `mean texture`.

In [6]:
from ray.data.preprocessors import StandardScaler

# Create a preprocessor to scale some columns
columns_to_scale = ["mean radius", "mean texture"]
preprocessor = StandardScaler(columns=columns_to_scale)

### 3a. Use AIR Trainers for supported ML frameworks
Use the Ray AIR trainer `XGBoostTrainer` with simple steps:
 1. define the parallelism for Ray compute
 2. define the XGBoost parameters for training
 3. supply the preprocessor for fitting and transforming dataset during training and validation
 4. provide the datasets for training and validation
 5. invoke `trainer.fit()` 
 
 <img src="../images/trainer.png" width="50%" height="25%">
 
 Simple API that does a lot behind the scenes for you!

In [9]:
from ray.air.config import ScalingConfig
from ray.train.xgboost import XGBoostTrainer

trainer = XGBoostTrainer(
    scaling_config=ScalingConfig(
        # Number of workers to use for data parallelism.
        num_workers=2,
        # Whether to use GPU acceleration.
        use_gpu=False),
    label_column="target",
    num_boost_round=20,
    params={
        # XGBoost specific params
        "objective": "binary:logistic",
        "eval_metric": ["logloss", "error"],
    },
    # our train and validation dataset and preprocessor
    datasets={"train": train_dataset, "valid": valid_dataset},
    preprocessor=preprocessor,
)

##### Fit the trainer

In [10]:
result = trainer.fit()
# print(result.metrics)

(_RemoteRayXGBoostActor pid=92178) [13:47:20] task [xgboost.ray]:4808387936 got new rank 1
(_RemoteRayXGBoostActor pid=92177) [13:47:20] task [xgboost.ray]:4755840448 got new rank 0


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train-error,train-logloss,training_iteration,trial_id,valid-error,valid-logloss,warmup_time
XGBoostTrainer_633df_00000,2022-10-14_13-47-22,True,,1e78f8922f2342bf880d0f73050358de,0,Juless-MacBook-Pro-16,21,127.0.0.1,92157,True,4.99818,0.6509,4.99818,1665780442,0,,0,0.0184957,21,633df_00000,0.0409357,0.0893879,0.00550771


### 3b. Use AIR Tuner for hyperparameter search

What if you want to do hyperparameter optimization during training and use the best config for the model? Well, you can then use Tuner and supply your training function, Trainer, as part of the argument, along 
with other Tuner configuration. 

<img src="../images/tuner.png" width="50%" height="25%">
Again, simple steps:
 1. define your hyperparameter space
 2. define `TuneConfig` for number of trials and parallelism 
 3. invoke `tuner.fit()`

In [11]:
from ray import tune

param_space = {"params": {"max_depth": tune.randint(1, 9)}}
metric = "train-logloss"
our_mode="min"

In [12]:
from ray.tune.tuner import Tuner, TuneConfig
from ray.air.config import RunConfig

tuner = Tuner(
    trainer,
    param_space=param_space,
    tune_config=TuneConfig(num_samples=5, metric=metric, mode=our_mode),
)
# Execute tuning.
result_grid = tuner.fit()

(_RemoteRayXGBoostActor pid=92698) [13:51:29] task [xgboost.ray]:4901727584 got new rank 1
(_RemoteRayXGBoostActor pid=92697) [13:51:29] task [xgboost.ray]:4877151584 got new rank 0


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train-error,train-logloss,training_iteration,trial_id,valid-error,valid-logloss,warmup_time
XGBoostTrainer_f84e5_00000,2022-10-14_13-51-31,True,,38b9027795a94b0fa10e7609135f7b87,0_max_depth=1,Juless-MacBook-Pro-16,21,127.0.0.1,92681,True,3.99127,0.648581,3.99127,1665780691,0,,0.0175879,0.0955215,21,f84e5_00000,0.0292398,0.112007,0.00591516
XGBoostTrainer_f84e5_00001,2022-10-14_13-51-33,True,,e9aa8736e89040c9820818fbab30abd6,1_max_depth=6,Juless-MacBook-Pro-16,21,127.0.0.1,92691,True,5.08396,0.865617,5.08396,1665780693,0,,0,0.0184957,21,f84e5_00001,0.0409357,0.0893879,0.00583005
XGBoostTrainer_f84e5_00002,2022-10-14_13-51-34,True,,3143aa960bac43c8bee8e6cb324f09f9,2_max_depth=6,Juless-MacBook-Pro-16,21,127.0.0.1,92692,True,5.10759,0.895789,5.10759,1665780694,0,,0,0.0184957,21,f84e5_00002,0.0409357,0.0893879,0.00528216
XGBoostTrainer_f84e5_00003,2022-10-14_13-51-38,True,,2bf046e02ab841bfbbf025600ddd8edf,3_max_depth=3,Juless-MacBook-Pro-16,21,127.0.0.1,92752,True,5.00233,0.894817,5.00233,1665780698,0,,0,0.0215151,21,f84e5_00003,0.0350877,0.0765915,0.00592518
XGBoostTrainer_f84e5_00004,2022-10-14_13-51-39,True,,1faf62ec280e45729cbd9aae10a9206c,4_max_depth=1,Juless-MacBook-Pro-16,21,127.0.0.1,92783,True,2.98231,0.0774159,2.98231,1665780699,0,,0.0175879,0.0955215,21,f84e5_00004,0.0292398,0.112007,0.00568128


(_RemoteRayXGBoostActor pid=92712) [13:51:31] task [xgboost.ray]:5132070240 got new rank 1
(_RemoteRayXGBoostActor pid=92711) [13:51:31] task [xgboost.ray]:4899118576 got new rank 0
(_RemoteRayXGBoostActor pid=92716) [13:51:31] task [xgboost.ray]:5013737040 got new rank 1
(_RemoteRayXGBoostActor pid=92715) [13:51:31] task [xgboost.ray]:5240561136 got new rank 0
(_RemoteRayXGBoostActor pid=92765) [13:51:35] task [xgboost.ray]:5120585008 got new rank 1
(_RemoteRayXGBoostActor pid=92764) [13:51:35] task [xgboost.ray]:4725759520 got new rank 0
(_RemoteRayXGBoostActor pid=92789) [13:51:37] task [xgboost.ray]:4764458576 got new rank 0
(_RemoteRayXGBoostActor pid=92790) [13:51:37] task [xgboost.ray]:4757744752 got new rank 1


In [13]:
# Fetch the best result with its best hyperparameter config 
best_result = result_grid.get_best_result()
print("Best Result:", best_result)

Best Result: Result(metrics={'train-logloss': 0.01849572773292735, 'train-error': 0.0, 'valid-logloss': 0.08938791319913073, 'valid-error': 0.04093567251461988, 'should_checkpoint': True, 'done': True, 'trial_id': 'f84e5_00001', 'experiment_tag': '1_max_depth=6'}, error=None, log_dir=PosixPath('/Users/jules/ray_results/XGBoostTrainer_2022-10-14_13-51-26/XGBoostTrainer_f84e5_00001_1_max_depth=6_2022-10-14_13-51-27'))


### Ray AIR Checkpoints

The AIR trainers, tuners, and custom pretrained model generate Checkpoints. An AIR Checkpoint is a format for models that are used across different components of the Ray AI Runtime. This common format allows easy interoperability among AIR components and seamless integration with external supported machine learning frameworks. Read more
about [Checkpoints]().

<img src="../images/checkpoints.jpeg" height="25%" and width="50%"> 

### 4. Use AIR `BatchPreditor` for batch prediction
Once you have trained and tuned your model, create a batch predictor from best model using the `best_result.checkpoint` and do batch inference. 

<img src="../images/batch_predictor.png" height="25%" and width="50%"> 

In [15]:
from ray.train.batch_predictor import BatchPredictor
from ray.train.xgboost import XGBoostPredictor

batch_predictor = BatchPredictor.from_checkpoint(best_result.checkpoint, XGBoostPredictor)

predicted_probabilities = batch_predictor.predict(test_dataset)
print("PREDICTED PROBABILITIES")
predicted_probabilities.show()

Map Progress (1 actors 0 pending): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.49it/s]

PREDICTED PROBABILITIES
{'predictions': 0.9964648485183716}
{'predictions': 0.9951295852661133}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9964648485183716}
{'predictions': 0.9969868063926697}
{'predictions': 0.9947494864463806}
{'predictions': 0.9899886250495911}
{'predictions': 0.9952162504196167}
{'predictions': 0.3375702202320099}
{'predictions': 0.9766711592674255}
{'predictions': 0.0037899704184383154}
{'predictions': 0.9948934316635132}
{'predictions': 0.9472665786743164}
{'predictions': 0.989780068397522}
{'predictions': 0.9952002763748169}
{'predictions': 0.18953870236873627}
{'predictions': 0.2149435132741928}
{'predictions': 0.99428790807724}
{'predictions': 0.9890844225883484}
{'predictions': 0.0037899704184383154}


### 5. Use `PredictorDeployment` for online inference

Deploy the best model as an inference service by using Ray Serve and the `PredictorDeployment` class.

<img src="../images/online_predictor.png" height="25%" and width="50%">

In [16]:
from ray import serve
from fastapi import Request
from ray.serve import PredictorDeployment
from ray.serve.http_adapters import pandas_read_json

serve.run(
    PredictorDeployment.options(name="XGBoostService", num_replicas=2, route_prefix="/rayair").bind(
        XGBoostPredictor, result.checkpoint, http_adapter=pandas_read_json
    )
)

(ServeController pid=93079) INFO 2022-10-14 13:54:53,208 controller 93079 http_state.py:129 - Starting HTTP proxy with name 'SERVE_CONTROLLER_ACTOR:SERVE_PROXY_ACTOR-b74df5b303e952699599aa9660d7392271d77abc0ae5c132334378e2' on node 'b74df5b303e952699599aa9660d7392271d77abc0ae5c132334378e2' listening on '127.0.0.1:8000'
(HTTPProxyActor pid=93081) INFO:     Started server process [93081]
(ServeController pid=93079) INFO 2022-10-14 13:54:53,927 controller 93079 deployment_state.py:1310 - Adding 2 replicas to deployment 'XGBoostService'.


RayServeSyncHandle(deployment='XGBoostService')

After deploying the service, you can send requests to it.

In [17]:
import requests

sample_input = test_dataset.take(1)
sample_input = dict(sample_input[0])

output = requests.post("http://localhost:8000/rayair", json=[sample_input]).json()
print(output)

[{'predictions': 0.9964648485183716}]


(HTTPProxyActor pid=93081) INFO 2022-10-14 13:54:59,008 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 307 11.4ms
(HTTPProxyActor pid=93081) INFO 2022-10-14 13:54:59,029 http_proxy 127.0.0.1 http_proxy.py:315 - POST /rayair 200 16.8ms
(ServeReplica:XGBoostService pid=93084) INFO 2022-10-14 13:54:59,028 XGBoostService XGBoostService#bukdMN replica.py:505 - HANDLE __call__ OK 13.4ms
(ServeReplica:XGBoostService pid=93083) INFO 2022-10-14 13:54:59,006 XGBoostService XGBoostService#zKpBjc replica.py:505 - HANDLE __call__ OK 0.3ms


In [18]:
ray.shutdown()

### Homework

1. Have a go at Ray AIR examples in the documentation.

 📖 [Back to Table of Contents](./ex_00_tutorial_overview.ipynb)<br>
⬅️ [Previous notebook](./ex_07_ray_data.ipynb) <br>

Done! 🍻
 